In [1]:
from config import *

# Output Parsers

In [12]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [13]:
model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [14]:
# Define your desired data structure
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, fields):
        if fields[-1] != "?":
            raise ValueError("Badly formatted question!")
        return fields

In [15]:
# Set up a parser + inject instuctions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [16]:
prompt = PromptTemplate(
    template="Answer the user query. \n {format_instructions}\n {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [19]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [20]:
output = model(_input.to_string())

In [21]:
parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

# List Parser

In [2]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

In [5]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

In [6]:
model = OpenAI(temperature=0)

In [7]:
_input = prompt.format(subject="ice cream flavors")
output = model(_input)

In [8]:
output_parser.parse(output)

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Mint Chocolate Chip',
 'Cookies and Cream']

# Datetime Parser

In [33]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [34]:
output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}

"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [35]:
chain = LLMChain(prompt=prompt, llm=OpenAI())

In [36]:
output = chain.run("around when was bitcoin founded?")

In [38]:
output_parser.parse(output)

datetime.datetime(2008, 1, 3, 18, 15, 5)

# Enum Parser

In [40]:
from langchain.output_parsers.enum import EnumOutputParser

In [41]:
from enum import Enum

class Colors(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"

In [42]:
parser = EnumOutputParser(enum=Colors)

In [43]:
parser.parse("red")

<Colors.RED: 'red'>

In [45]:
# can handle spaces
parser.parse(" green")

<Colors.GREEN: 'green'>

In [47]:
# and new lines

parser.parse("blue\n")

<Colors.BLUE: 'blue'>

In [48]:
# And raises errors when appropriate

parser.parse("yellow")

OutputParserException: Response 'yellow' is not one of the expected values: ['red', 'green', 'blue']

# Auto-fixing Parser

In [59]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [60]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
    
actor_query = "Generate the filmography for a random actor."
parser = PydanticOutputParser(pydantic_object=Actor)

In [61]:
misformatted = "{'name': 'Tom Hanks', 'film_names':['Forrest Gump']}"

In [62]:
parser.parse(misformatted)

OutputParserException: Failed to parse Actor from completion {'name': 'Tom Hanks', 'film_names':['Forrest Gump']}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [63]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [65]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

# Pydantic (JSON) Parser

In [66]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [68]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [69]:
model_name="text-davinci-003"
temperature=0.0
model=OpenAI(model_name=model_name, temperature=temperature)

In [72]:
# Define you desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to setup a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("question must end with a question mark")
        return field

joke_query = "Tell me a joke."    

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query\n{format_instructions}\n{query}\n",
    input_variables=['query'],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [74]:
# Here's another example, but with a comppoud typed field.

class Actor(BaseModel):
    name: str = Field(description="The actor's name")
    film_names: List[str] = Field(description="The names of the films the actor has been in")
    
actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n\n{query}\n",
    input_variables=['query'],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

output = model(_input.to_string())

parser.parse(output)

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Saving Private Ryan', 'The Green Mile', 'Cast Away', 'Toy Story'])

# Retry Parser

In [75]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)

from pydantic import BaseModel

from typing import List, Optional

In [76]:
template = """
Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:
"""

In [77]:
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to action")
    
parser = PydanticOutputParser(pydantic_object=Action)

In [78]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [79]:
prompt_value = prompt.format_prompt(query="who is leo di caprio gf?")

In [94]:
bad_response = '{"action": "Search"}'

In [82]:
# If we try to parse this response as is, we will get an error
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

In [84]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [95]:
fix_parser.parse(bad_response)

Action(action='Search', action_input='keyword')

In [96]:
from langchain.output_parsers import RetryWithErrorOutputParser

In [98]:
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [99]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='Search', action_input='Leo DiCaprio Girlfriend')

# Structured Output Parser

In [100]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [101]:
ResponseSchema = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]

output_parser = StructuredOutputParser.from_response_schemas(ResponseSchema)

In [108]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the user question as best as possible.\n {format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [109]:
model = OpenAI(temperature=0)

In [110]:
_input = prompt.format_prompt(question="what's the capital of france?")
output = model(_input.to_string())

In [111]:
output_parser.parse(output)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/France'}

And here's an example of using this in a chat model.

In [112]:
Chat_model = ChatOpenAI(temperature=0)

In [116]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible. \n{format_instructions}\n{question}")
    ],
    input_variables = ['question'],
    partial_variables = {"format_instructions": format_instructions}
)

In [118]:
_input = prompt.format_prompt(question="what's the capital of france?")
output = Chat_model(_input.to_messages())

In [119]:
output_parser.parse(output.content)

{'answer': 'The capital of France is Paris.',
 'source': 'https://en.wikipedia.org/wiki/Paris'}